# 8. Dashboard

In this notebook we created a dashboard based on a little EDA with the tmdb dataset. As output we decided, the dashboard should contain a figure with the most expensive movies, the most popular ones, the proportion of different movie genres, the production countries and the releases over the year. As common for a dashboard the graphs should have functionalities. We decided to give te user the choice for which year he/she/they wants to have those informations.

The dashboard was deployed on heroku an can be found [here](https://dashboard-movie.herokuapp.com/)

In [176]:
# load requirements
import pandas as pd

import dash
from dash import dcc
from dash import html
import plotly.express as px
from ast import literal_eval
from dash import dash_table

from dash.dependencies import Input, Output


In [177]:
smd = pd.read_csv("data/data_for_dash.csv")

In [178]:
smd.describe()

,popularity,year,id,budget
count,9653.000000,9653.000000,9653.000000,9.653000e+03
mean,7.631171,1994.399151,50671.254221,1.699881e+07
std,10.828585,18.540407,85041.392581,3.375294e+07
min,0.000004,1902.000000,2.000000,0.000000e+00
25%,3.188634,1987.000000,9612.000000,0.000000e+00
50%,6.573364,1999.000000,16219.000000,2.400000e+05
75%,9.989679,2007.000000,42618.000000,2.000000e+07
max,547.488298,2017.000000,461805.000000,3.800000e+08


In [179]:
smd_year.sort_values("popularity",ascending =False).popularity[:10]

0      21.946943
97     20.755149
123    18.761467
43     18.457430
5      17.924927
163    17.502375
1      17.015539
176    16.885184
46     16.302466
203    16.236745
Name: popularity, dtype: float64

In [180]:
# this function converts lists within the strings in the specified columns into real lists
columns = ['actors', "genre", "production_country"]
for i in columns:
    smd[i] = smd[i].apply(literal_eval)



#this function coverts the series data into an 1D array
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])


In [181]:
#functions for dashboard figures

# get year values for dropdown-menu
years = smd.year.unique()
smd_year = smd[smd["year"] == years[0]]

# dat subsets for plots
smd_ya = pd.DataFrame(to_1D(smd_year["actors"]).value_counts())
smd_yg = pd.DataFrame(to_1D(smd_year["genre"]).value_counts())
smd_yp = pd.DataFrame(to_1D(smd_year["production_country"]).value_counts())
rel = pd.DataFrame(smd_year.month.value_counts().reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']))

# figures 

fig1 = px.bar(x=smd_year.sort_values("budget")["original_title"][:10], y=smd_year.sort_values("budget").index[:10])
fig1.update_layout(title_text='Most expensive movies', title_x=0.5)

fig2 = px.bar(x=smd_year.sort_values("popularity")["original_title"][:10], y=smd_year.sort_values("popularity").index[:10])
fig2.update_layout(title_text='Most popular movies', title_x=0.5)

fig3 = px.pie(values=smd_ya[0][:10], names=smd_yg.index[:10])
fig3.update_layout(title_text='Movie genres', title_x=0.5)

fig4 = px.choropleth(locations=smd_yp.index,
                    locationmode="country names",
                    color = smd_yp[0])
fig4.update_layout(title_text='Movies produced by Country', title_x=0.5)

fig5 = px.line(x=rel.index, y=rel["month"])
fig5.update_layout(title_text='Releases over the year', title_x=0.5)

#

In [185]:
app =dash.Dash()
app.title = 'Mokey Dash'

app.layout = html.Div(style = {"background-color": "white"},children=[html.Div([
                                                                    html.H2("Please choose a year you want to have information about",style = {"padding-top": "10px","padding-bottom": "10px","background-color": "#FFF1AF"}),
                                                                    dcc.Dropdown(
                                                                                id="dropdown",
                                                                                options=sorted([{'label': i, 'value': i} for i in years], key = lambda x: x['label']),
                                                                                value=years[0],
                                                                                clearable=False, 
                                                                                style= {"width": "50%",'margin':'auto'}),
                                                                    dcc.Graph(id="exp-movies", figure = fig1,style = {"width":"30%", "margin-left": "20px","position": "relative", "display":"inline-block"}),
                                                                    dcc.Graph(id="pop-movies", figure = fig2,style = {"width":"30%", "margin-left": "20px","position": "relative","display":"inline-block"}),
                                                                    dcc.Graph(id="pieplot", figure = fig3,style = {"width":"30%", "margin-left": "20px","position": "relative","display":"inline-block"}),
                                                                    dcc.Graph(id="mapplot", figure = fig4,style = {"width":"65%", "margin-left":"20px","position": "relative","display":"inline-block"}),
                                                                    dcc.Graph(id="lineplot", figure = fig5,style = {"width":"30%", "margin-left": "20px","position": "relative", "display":"inline-block"}),

                                                                    ])
                                                                ])



@app.callback(
    [Output("exp-movies", "figure"), Output("pop-movies", "figure"),Output("pieplot", "figure"), Output("mapplot", "figure"), Output("lineplot", "figure")], 
    [Input("dropdown", "value")])
def update_plots(year):
    smd_year = smd[smd["year"] == year]
    smd_ya = pd.DataFrame(to_1D(smd_year["actors"]).value_counts())
    smd_yg = pd.DataFrame(to_1D(smd_year["genre"]).value_counts())
    smd_yp = pd.DataFrame(to_1D(smd_year["production_country"]).value_counts())
    rel = pd.DataFrame(smd_year.month.value_counts().reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']))
    fig1 = px.bar(x=smd_year.sort_values("budget",ascending =False).budget[:10], y=smd_year.sort_values("budget", ascending =False)["original_title"][:10]).update_layout(title_text='Most expensive movies', title_x=0.5).update_xaxes(title="US $")
    fig1.update_yaxes(title=None, autorange="reversed")
    fig2 = px.bar(x=smd_year.sort_values("popularity",ascending =False).popularity[:10], y=smd_year.sort_values("popularity",ascending =False)["original_title"][:10]).update_layout(title_text='Most popular movies', title_x=0.5).update_xaxes(title="TMDB popularity score")
    fig2.update_yaxes(title=None, autorange="reversed")
    fig3 = px.pie(values=smd_ya[0][:10], names=smd_yg.index[:10]).update_layout(title_text='Movie genres', title_x=0.5)       
    fig4 = px.choropleth(locations=smd_yp.index,
                    locationmode="country names",
                    color = smd_yp[0]).update_layout(title_text='Movies produced by Country', title_x=0.5)
    fig5 = px.line(x=rel.index, y=rel["month"], markers = True).update_layout(title_text='Releases over the year', title_x=0.5).update_xaxes(title=None)
    fig5.update_yaxes(title=None)


    return fig1, fig2, fig3, fig4, fig5


                  
                   
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /assets/style.css?m=1639315388.1637542 HTTP/1.1" 304 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1638463670.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1638463670.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1638463670.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1638463670.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1638463670.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 11:02:30] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.